In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from gensim.models import Word2Vec


In [ ]:
from google.colab import drive

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/dataset/train-balanced-sarcasm.csv')

In [ ]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [ ]:
# Preprocess the labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
# Tokenization and padding
max_words = 5000
max_len = 100

In [ ]:
df = df.dropna(subset=['comment'])


In [ ]:
# Ensure all comments are strings
df['comment'] = df['comment'].astype(str)



In [ ]:
# Preprocess the labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [ ]:
# Tokenization and padding
max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(df['comment'])
sequences = tokenizer.texts_to_sequences(df['comment'])
word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=max_len)

In [ ]:
# Prepare the Word2Vec model
word2vec_model = Word2Vec(sentences=[text.split() for text in df['comment']], vector_size=100, window=5, min_count=1, workers=4)
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(data, df['label'], test_size=0.2, random_state=42)

In [ ]:
# Convert labels to categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [ ]:
# Build the GRU model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          18058400  
                                                                 
 spatial_dropout1d_1 (Spati  (None, 100, 100)          0         
 alDropout1D)                                                    
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 18119202 (69.12 MB)
Trainable params: 60802 (237.51 KB)
Non-trainable params: 18058400 (68.89 MB)
_________________________________________________________________
None


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=4, batch_size=64, validation_data=(X_val, y_val), verbose=1)

Epoch 1/10
11916/12635 [===========================>..] - ETA: 3:09 - loss: 0.5922 - accuracy: 0.6794